<a href="https://colab.research.google.com/github/rathorebharat/mftracker/blob/main/mutliple-indicators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# To install the mftool library
!pip install mftool==2.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mftool: filename=mftool-2.8-py3-none-any.whl size=116617 sha256=81a774f7af7925278631d93ce26206c4536e79b0c55d2115540b2a42a8e1427f
  Stored in directory: /root/.cache/pip/wheels/b6/b3/af/5751d41db5452227e221aaca7f1ec823caa51ae7d40fb9dbde
Successfully built mftool


In [3]:
!pip install nsepy==0.8

  Preparing metadata (setup.py) ... done
  Created wheel for nsepy: filename=nsepy-0.8-py3-none-any.whl size=36056 sha256=c5277b40f70142350597bb0becc247e91682a035b118d26ba819e2c80bdf1d66
  Stored in directory: /root/.cache/pip/wheels/4c/87/cb/acaf83f625e5fc73e1fe6e2a8e97680c74cd72391850ef5a86
Successfully built nsepy


In [91]:
# https://medium.com/@TejasEkawade/getting-and-analyzing-mutual-funds-in-python-c2d0feb09881

# List of all mutual funds and their latest NAV
# https://www.amfiindia.com/spages/NAVAll.txt
# 148725;INF204KB19Z5;INF204KC1014;NIPPON INDIA NIFTY MIDCAP 150 INDEX FUND - DIRECT Plan - IDCW Option;25.3024;30-Aug-2024
# 138528;INF223J01NF2;-;PGIM India Global Equity Opportunities Fund - Direct Plan - Growth;47.78;30-Aug-2024
# 127042;INF247L01445;-;Motilal Oswal Midcap Fund-Direct Plan-Growth Option;115.7794;30-Aug-2024

import requests
search_string = 'HSBC Asia Pacific'  # Replace with the string to search for
url = 'https://www.amfiindia.com/spages/NAVAll.txt'  # Replace with the actual URL
response = requests.get(url)
page_content = response.text
for line in page_content.splitlines():
    if search_string in line:
        print(line)

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from mftool import Mftool
import yfinance as yf


mf = Mftool()
# result = mf.get_available_schemes('hdfc')
# for scheme_code, scheme in result.items():
#     print(scheme_code, scheme)

# Mutual Funds Analysis — Implementation
# Part 1: Tabular Representation
# Let’s pull data for multiple funds and save them as dataframes. These data frames will be saved in a dictionary.

end_date = datetime.date.today()

def fetch_mutual_fund_data(mutual_fund_code):
    mf = Mftool()

    df = (mf.get_scheme_historical_nav(mutual_fund_code,as_Dataframe=True).reset_index()\
          .assign(nav=lambda x: x['nav'].astype(float),
                 date=lambda x: pd.to_datetime(x['date'], format='%d-%m-%Y'))\
          .sort_values('date')\
          .reset_index(drop=True)\
         )
    return df

def fetch_ticker(ticker_code):
    # Download historical data using yfinance
    #df = yf.download(ticker_code, period="1y").reset_index()
    df = yf.download(ticker_code,start="2023-01-01", end="2024-11-14").reset_index()
    # Extract 'Open' values without multi-level index
    nav_values = df['Open'][ticker_code]  # Access 'Open' and then 'MOVE'
    df = pd.DataFrame({
    'date': pd.to_datetime(df.index, unit='D'),  # Assuming index represents dates
    'nav': nav_values  # Assign the extracted 'Open' values to 'Nav'
    })
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
    df = (df.assign(nav=lambda x: x['nav'].astype(float),
          date=lambda x: pd.to_datetime(x['date'], format='%Y-%m-%d'))\
      .sort_values('date')\
      .reset_index(drop=True)\
      )
    return df

# Fetch data for these funds
mutual_funds = {
    '119063': 'Nifty 50'
}

# Define the ticker symbol for the MOVE index
move_ticker = "^GSPC"

mf_data_dict = dict()

for mutual_fund_code, mutual_fund_desc in mutual_funds.items():
    print(mutual_fund_desc)
    mf_data_dict[mutual_fund_desc] = fetch_mutual_fund_data(mutual_fund_code)
# Next, create a function that computes the cumulative returns and CAGR. We are going to do this for 1 year, 2 years, and 3 years.

print(move_ticker)
mf_data_dict[move_ticker] = fetch_ticker(move_ticker)

def get_cumulative_returns(df,
                         nav_col='CLOSE',
                         date_col='date',
                         starting_date='1-1-2019',
                         ending_date = end_date):
    start_date = pd.to_datetime(starting_date, format='%d-%m-%Y')
    end_date = pd.to_datetime(ending_date, format='%d-%m-%Y')

    df = (df\
          .sort_values(date_col)\
          .query(f"{date_col} >= @start_date and {date_col} <=@end_date")\
          .assign(daily_returns=lambda x: x[nav_col].pct_change(),
                  cumulative_daily_returns=lambda x: (x['daily_returns'] + 1).cumprod())\
          .reset_index(drop=True)\
         )
    return df

# We now pass our data — mutual funds and benchmark index — through the function to calculate metrics.

time_periods = {
    1: '1-1-2024'
}
results = []
time_period_dataframes = {time_period: dict() for time_period in time_periods}

# Mutual Funds data
mutual_funds_subset = [
    'Nifty 50',
    '^GSPC'
]
for instrument_desc in mutual_funds_subset:
    for time_period, starting_date in time_periods.items():
        n_years = time_period
        mutual_fund_data = mf_data_dict[instrument_desc].copy(deep=True)
        #print(mutual_fund_data)
        print(mutual_fund_data.head())
        mf_with_cumulative = get_cumulative_returns(mutual_fund_data,
                                                           nav_col='nav',
                                                           date_col='date',
                                                           starting_date=starting_date,
                                                           ending_date = end_date)
        print(mf_with_cumulative)
        absolute_returns_prcnt = (mf_with_cumulative['cumulative_daily_returns'].values[-1] - 1) * 100
        cagr = ((mf_with_cumulative['nav'].iloc[-1]/mf_with_cumulative['nav'].iloc[0]) ** (1/n_years) - 1) * 100

        time_period_dataframes[time_period][instrument_desc] = mf_with_cumulative
        results += [(instrument_desc, time_period, absolute_returns_prcnt, cagr)]

# The results will be converted into a data frame.

result_df = pd.DataFrame(results, columns=['Fund', 'Years', 'Abs_returns_prcnt', 'CAGR']).\
pivot(index='Fund', columns='Years', values=['Abs_returns_prcnt', 'CAGR'])
result_df.columns = [f'{x}_{y}' for x, y in result_df.columns]
result_df = result_df.map(lambda x: f"{x:.2f}%")

time_period = 1

for idx, (instrument_desc, instrument_data) in enumerate(time_period_dataframes[time_period].items()):
    if idx == 0:
        ax = instrument_data.plot(y='cumulative_daily_returns', x='date', label=instrument_desc)
    else:
        instrument_data.plot(ax = ax, y='cumulative_daily_returns', x='date', label=instrument_desc)

plt.title(f"Cumulative Returns - {time_periods[time_period]}-year comparison")
ax.legend(loc='upper left')
plt.plot()

127073;INF336L01HC7;-;HSBC Asia Pacific (Ex Japan) Dividend Yield Fund - Growth;21.6861;13-Nov-2024
127071;INF336L01HF0;-;HSBC Asia Pacific (Ex Japan) Dividend Yield Fund - Growth Direct;23.3476;13-Nov-2024
127072;INF336L01HG8;INF336L01HH6;HSBC Asia Pacific IDCW Yield Fund Direct Plan IDCW;17.9894;13-Nov-2024
127070;INF336L01HD5;INF336L01HE3;HSBC Asia Pacific IDCW Yield Fund IDCW;18.1176;13-Nov-2024
Nifty 50


[*********************100%***********************]  1 of 1 completed

^GSPC
        date      nav  dayChange
0 2013-01-01  51.3793        NaN
1 2013-01-02  51.7411     0.3618
2 2013-01-03  51.8788     0.1377
3 2013-01-04  51.9355     0.0567
4 2013-01-07  51.6972    -0.2383
          date       nav  dayChange  daily_returns  cumulative_daily_returns
0   2024-01-01  207.3522     0.0971            NaN                       NaN
1   2024-01-02  206.6255    -0.7267      -0.003505                  0.996495
2   2024-01-03  205.2085    -1.4170      -0.006858                  0.989662
3   2024-01-04  206.5547     1.3462       0.006560                  0.996154
4   2024-01-05  207.0511     0.4964       0.002403                  0.998548
..         ...       ...        ...            ...                       ...
210 2024-11-07  232.8305    -2.7229      -0.011560                  1.122875
211 2024-11-08  232.3380    -0.4925      -0.002115                  1.120499
212 2024-11-11  232.2676    -0.0704      -0.000303                  1.120160
213 2024-11-12  229.7877  

IndexError: index -1 is out of bounds for axis 0 with size 0

In [84]:
print (mf_data_dict['^MOVE'])
print (mf_data_dict['Nifty 50'])

          date         nav
0   1970-01-01  121.610001
1   1970-01-02  129.970001
2   1970-01-03  125.779999
3   1970-01-04  119.529999
4   1970-01-05  113.870003
..         ...         ...
462 1971-04-08  130.429993
463 1971-04-09  117.639999
464 1971-04-10  104.080002
465 1971-04-11   99.849998
466 1971-04-12    0.000000

[467 rows x 2 columns]
           date       nav  dayChange
0    2013-01-01   51.3793        NaN
1    2013-01-02   51.7411     0.3618
2    2013-01-03   51.8788     0.1377
3    2013-01-04   51.9355     0.0567
4    2013-01-07   51.6972    -0.2383
...         ...       ...        ...
2916 2024-11-07  232.8305    -2.7229
2917 2024-11-08  232.3380    -0.4925
2918 2024-11-11  232.2676    -0.0704
2919 2024-11-12  229.7877    -2.4799
2920 2024-11-13  226.6671    -3.1206

[2921 rows x 3 columns]


        date      nav  dayChange
0 2013-01-01  51.3793        NaN
1 2013-01-02  51.7411     0.3618
2 2013-01-03  51.8788     0.1377
3 2013-01-04  51.9355     0.0567
4 2013-01-07  51.6972    -0.2383
          date       nav  dayChange  daily_returns  cumulative_daily_returns
0   2024-01-01  207.3522     0.0971            NaN                       NaN
1   2024-01-02  206.6255    -0.7267      -0.003505                  0.996495
2   2024-01-03  205.2085    -1.4170      -0.006858                  0.989662
3   2024-01-04  206.5547     1.3462       0.006560                  0.996154
4   2024-01-05  207.0511     0.4964       0.002403                  0.998548
..         ...       ...        ...            ...                       ...
210 2024-11-07  232.8305    -2.7229      -0.011560                  1.122875
211 2024-11-08  232.3380    -0.4925      -0.002115                  1.120499
212 2024-11-11  232.2676    -0.0704      -0.000303                  1.120160
213 2024-11-12  229.7877    -2.4

IndexError: index -1 is out of bounds for axis 0 with size 0